# Lab 4.2 - Chatbot for question-answering

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

In this notebook, we will create a Telegram chatbot that will answer questions, given a predefined question-answer set. 

**Main goal of this notebook**: The most important goal of this notebook is to have a Telegram chatbot that you can ask factual questions to, and receive predefined answers.

**At the end of this notebook, you will**:
* **Use a predefined question - answering dataset**: 

## Using a question-answer dictionary

In [1]:
import json
import random
import datetime
import requests

In [2]:
def read_token():
    tokens_path = './data/tokens.json'
    with open(tokens_path) as f:
        tokens = json.load(f)

    return tokens['CLTL_token']

def read_qa():
    qa_path = './data/q-and-a.json'
    with open(qa_path) as f:
        qa_data = json.load(f)

    return qa_data

In [3]:
class BotHandler:
    def __init__(self, token):
        self.token = token
        self.api_url = "https://api.telegram.org/bot{}/".format(token)

    def get_all_messages(self, offset=None, timeout=200):
        """ Function to get all messages sent to the bot """
        method = 'getUpdates'
        params = {'timeout': timeout, 'offset': offset}
        resp = requests.get(self.api_url + method, params)
        return resp.json()['result']

    def get_last_message_by(self, chat_id):
        """ Function to get the last message sent to the bot by a specific user"""
        messages = self.get_all_messages()
        messages_by_user = list(filter(lambda d: d['message']['chat']['id'] == chat_id, messages))

        if messages_by_user:
            last_message = messages_by_user[-1]['message']['text']
        else:
            last_message = None

        return last_message
    
    def send_message_to(self, chat_id, text):
        """ Function to send a message from the bot to a specific user"""
        params = {'chat_id': chat_id, 'text': text}
        method = 'sendMessage'
        resp = requests.post(self.api_url + method, params)

In [4]:
CLTL_TOKEN = read_token()
qa_data = read_qa()
user_id = 408043639
print(json.dumps(qa_data['intents'][0], indent=2))

{
  "category": "greeting",
  "questions": [
    "Hi there",
    "How are you",
    "Is anyone there",
    "Hey",
    "Hola",
    "Hello",
    "Good day"
  ],
  "responses": [
    "Hello",
    "Good to see you again",
    "Hi there, how can I help"
  ]
}


In [5]:
def create_response(message, qa_data):
    response = "I cannot respond to this"
    
    for i in qa_data['intents']:
        if message in i['questions']:
            response = random.choice(i['responses'])
            break
    
    return response

In [6]:
bot = BotHandler(CLTL_TOKEN)
last_message = bot.get_last_message_by(user_id)
response = create_response(last_message, qa_data)
bot.send_message_to(user_id, response)


print("Received: {message}".format(message=last_message))
print("Responded: {response}".format(response=response))

Received: Hi there
Responded: Good to see you again


## End of this notebook